### data cleansing
Data cleansing task aimed at documents from **antonio azeredo da silveira** collection. The image files went through ocr process, which provided text files with good quality but with the need of data cleansing process. 

We also added key terms which might be used on future tasks.

Info:
* number of documents: ~10.000
* number of files: 66.938
* number of files: 66.938

In [1]:
import re
import enchant
import os
import sys
import glob
import string
import time
import operator
from collections import OrderedDict, Counter
from IPython.display import clear_output
#%pylab inline

### set user specific variables

In [3]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Verifica qual sistema operacional está sendo usado e cria variáveis. Renato = Linux ; Marcelo = nt (Windows)
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
if os.name == 'nt':
    encoding_type = 'utf-8'
    path_input = 'C:/Users/marcelo.ribeiro/Documents/textfiles-original-renamed2/'
    path_output = 'C:/Users/marcelo.ribeiro/Documents/textfiles-corrected-test/'
else:
    encoding_type = 'ISO-8859-1'
    path_input = '../textfiles-original-renamed2/'
    path_output = '../textfiles-corrected-renamed2'

onlyfiles = [f for f in os.listdir(path_input) if os.path.isfile(os.path.join(path_input,f)) and not f.startswith('.')]
onlyfiles.sort()

In [4]:
onlyfiles[:4]

['AAS_mre_ag_1973.11.20_doc_I-100_pag1_img34229.txt',
 'AAS_mre_ag_1973.11.20_doc_I-101_pag1_img34230.txt',
 'AAS_mre_ag_1973.11.20_doc_I-102_pag1_img34231.txt',
 'AAS_mre_ag_1973.11.20_doc_I-103_pag1_img34232.txt']

### data cleansing on .txt files
* Uses regex to fix words. The most effective strategy was to focus on punctuation symbols. 
* There is a backup file with  observations for some lines. 
* The regex are organized in effectiveness order (as much as possible). 
* Some regex were used not for their effectiveness, but because they were extremely important information.
* Some stopwords were also corrected. Although they are not relevant for topic modelling, they are extremely important for entity extraction.

In [ ]:
percentil = int(len(onlyfiles)/100)
freqtotal = Counter()
for txt in onlyfiles:
    with open(os.path.join(path_in,txt), 'r', encoding='utf-8') as f:
        
        '''measures completion percentage ''' 
        count = onlyfiles.index(txt)
        if count % percentil == 0: print(int(count/percentil),'% done')
        if (count) % (percentil-1) == 0: clear_output()
        
        '''read files''' 
        text = f.read()
        text = text.lower()
        
        '''clean files''' 
        text = re.sub('—', r'-',text)
        text = re.sub('(?<!exm|.ex|..\d)ª\n*',r'a',text)
        text = re.sub('(?<!exm|.ex|..\d)ª',r'a',text)
        text = re.sub('nº\n*(\d)',r'nº \1',text)
        text = re.sub('(?<!exm|.ex|..\d)º\n*',r'o',text)
        text = re.sub('(?<!exm|.ex|..\d)º.\D',r'o',text)
        text = re.sub('([a-z])-\n([a-z])',r'\1\2',text)
        text = re.sub('([^\W\d])\»\n([^\W\d])',r'\1 \2',text, flags=re.I)
        text = re.sub('([^\W\d])\«\n([^\W\d])',r'\1 \2',text, flags=re.I)
        text = re.sub('([^\W\d])\"([^\W\d])',r'\1 \2',text, flags=re.I)
        text = re.sub('[\“\”\"\'\«\»]{2,}',r'"',text, flags=re.I)
        text = re.sub('(\“|\”|\«|\»)',r'"',text, flags=re.I)
        text = re.sub('([^\W\d])\«([^\W\d])',r'\1-\2',text, flags=re.I)
        text = re.sub('([^\W\d])\»([^\W\d])',r'\1 \2',text, flags=re.I)
        text = re.sub('([^\W\d_])\_([^\W\d_])',r'\1 \2',text, flags=re.I)
        text = re.sub('([^\W\d])\!([^\W\d])',r'\1i\2',text, flags=re.I)
        text = re.sub('([^\W\d])\;([áàaâã]o|[óòoôõ]e)',r'\1ç\2',text, flags=re.I)
        text = re.sub('([^\W\d])\;(os?\W)',r'\1ã\2',text, flags=re.I) 
        text = re.sub('([^\W\d])\;([^\W\d])',r'\1i\2',text, flags=re.I) 
        text = re.sub('([^\W\d])\:([^\W\d])',r'\1i\2',text, flags=re.I)
        text = re.sub('([^\W\d])\“([^\W\d])',r'\1n\2',text, flags=re.I)
        text = re.sub('“',r'"',text, flags=re.I)
        text = re.sub('([^\W\d])\”([^\W\d])',r'\1n\2',text, flags=re.I)
        text = re.sub('”',r'"',text, flags=re.I)
        text = re.sub('([^\W\d])\(([^\W\d])',r'\1 (\2',text, flags=re.I) 
        text = re.sub('([^\W\d])\)([^\W\d])',r'\1) \2',text, flags=re.I) 
        text = re.sub('([^\W\d])\$([^\W\d])',r'\1s\2',text, flags=re.I)
        text = re.sub('(\.)([^\s0-9])',r'\1 \2',text, flags=re.I)
        text = re.sub('(\,)([^\s0-9])',r'\1 \2',text, flags=re.I)
        text = re.sub('([^\W\d])1([^\W\d])',r'\1i\2',text, flags=re.I)
        text = re.sub('([^\W\d])1([^\W\d])',r'\1i\2',text, flags=re.I)
        text = re.sub('([^\W\d])1([^\/ªº][\D])',r'\1l\2',text)
        text = re.sub('([\D][^\/])1([^\W\d])',r'\1l\2',text)
        text = re.sub('([^\W\d])5([^\/ªº][\D])',r'\1s\2',text, flags=re.I)
        text = re.sub('([\D][^\/])5([^\W\d])',r'\1s\2',text, flags=re.I)
        text = re.sub('([^\W\d])0([^\/][\D])',r'\1o\2',text, flags=re.I)
        text = re.sub('([\D][^\/])0([^\W\d])',r'\1o\2',text, flags=re.I)
        text = re.sub('([\D][^\/])11([^\W\d])',r'\1li\2',text, flags=re.I)
        text = re.sub('(3)(rd)([^\w][^\d])',r'\1 \2\3',text, flags=re.I)
        text = re.sub('(2)(nd)([^\w][^\d])',r'\1 \2\3',text, flags=re.I)
        text = re.sub('(1)(st)([^\w][^\d])',r'\1 \2\3',text, flags=re.I)
        text = re.sub('([04-9])(th)([^\w][^\d])',r'\1 \2\3',text, flags=re.I)
        text = re.sub('n[áàaâ]oio', r'nacio',text, flags=re.I)
        text = re.sub('n[áàaâ]oue', r'naque',text, flags=re.I) 
        text = re.sub('n[áàâã]o(pro[l1]|ferro|al[íìiî]|[íìiî]nterv|nuc[l1]|d[íìiî]scr|exp[l1]|reco[nmh]|imp[l1]|gov|indu|redu|rec[íìiî]p|memb|aut|ren|s[íìiî]g|[íìiî]ng|tar|intro|ofi)', r'não-\1',text, flags=re.I)
        text = re.sub('anti(nuc|com|ame|trus|castr|col|cl[ií]|sem|eco|arm|per|apar|infl|disc|rac|chi|imp|sio|zio|sub|dum|eur|hig|dem|oci|port|tan|air|gov|pol|ter|sat|sov|ét|per|rus|a[eé]|res|mil|d[eé]t|fas|hit|bra|nav|cic|rec|isr)', r'anti-\1',text, flags=re.I)        
        text = re.sub('bras\w\we\w[^\w]{0,3}\n+r([ao]s?[^\w])', r'brasileir\1',text, flags=re.I)
        text = re.sub('([inaro])[^\w]{0,3}\n+c[íìiîl0-9]al([^\w])', r'\1cial\2',text, flags=re.I) 
        text = re.sub('([napi])[^\w]{0,3}\n+c[íìiîl0-9]ona[l1i]([^\w])', r'\1cional\2',text, flags=re.I) 
        text = re.sub('e[^\w]{0,3}\n+s[íìiîl0-9]den[íìiîl0-9]e([^\w])', r'esidente\1',text, flags=re.I)
        text = re.sub('m\w[^\w]{0,3}\n+n[íìiî]st\w([oa]s?[^\w])', r'ministr\1',text, flags=re.I)
        text = re.sub('([ai])[^\w]{0,3}\n+l[íìiî]dade([^\w])', r'\1lidade\2',text, flags=re.I)
        text = re.sub('([lora])[^\w]{0,3}\n+v[íìiî]men\wo([^\w])', r'\1vimento\2',text, flags=re.I)
        text = re.sub('([e])[^\w]{0,3}\n+senvolv[íìiî]men([^\w])', r'\1senvolvimen\2',text, flags=re.I)
        text = re.sub('(a\w)[^\w]{0,3}\n+gen\w[íìiî]n([oa]s?[^\w])', r'argentin\2',text, flags=re.I)
        text = re.sub('([^\w])pres[íìiî]den[^\w]{0,3}\n+([ct])', r'\1presiden\2',text, flags=re.I)
        text = re.sub('([^\w])desenvo\w[^\w]{0,3}\n+([vt])', r'\1desenvol\2',text, flags=re.I)
        text = re.sub('([^\w])gove\w[^\w]{0,3}\n+([aoim])', r'\1gover\2',text, flags=re.I) 
        text = re.sub('([a-z])[^a-z]ção', r'\1ção',text, flags=re.I)
        text = re.sub('(ç[áàâã]o|ções|ências|éias|érios|ível|ssão|ômic[ao]s|ípios)([^\W\d])',r'\1 \2',text, flags=re.I)
        text = re.sub('(ência|éia|ério|ômic[ao]|ípio)([^\W\ds])',r'\1 \2',text, flags=re.I)
        text = re.sub('([^a-z])n[áàaâã]o([^a-z])', r'\1não\2',text, flags=re.I)
        text = re.sub('([^a-z])pa[íiìî0-9]ses([^a-z])', r'\1países\2',text, flags=re.I)
        text = re.sub('rela[cç][óòoôõ\w]es', r'relações',text, flags=re.I)
        text = re.sub('brasil([eé][^a-z][^i])', r'brasil \1',text, flags=re.I)
        text = re.sub('exce[l1][èéê\w]nc[íiìî0-9]a', r'excelência',text, flags=re.I)
        text = re.sub('([^a-z])s[áàaâã]o([^a-z])', r'\1são\2',text, flags=re.I)
        text = re.sub('([^a-z])reuni[áàaâã]o([^a-z])', r'\1reunião\2',text, flags=re.I)
        text = re.sub('assemb[l1][èéeê][íiìî0-9]a', r'assembleia',text, flags=re.I)
        text = re.sub('brasilem', r'brasil em',text, flags=re.I)
        text = re.sub('minist[èéeê]r[íiìî0-9]o', r'ministério',text, flags=re.I)
        text = re.sub('c\w\wpe\wa[cç][aàââã]o', r'cooperação',text, flags=re.I)
        text = re.sub('com\wss[áàaâã]o([^a-z])', r'comissão\1',text, flags=re.I)
        text = re.sub('mat[èéeê]r[íiìîl0-9]a([^a-z])', r'matéria\1',text, flags=re.I)
        text = re.sub('([^a-z])quest[áàaâã]o([^a-z])', r'\1questão\2',text, flags=re.I)
        text = re.sub('([^a-z])rela[cç][áàaâã]o([^a-z])', r'\1relação\2',text, flags=re.I)
        text = re.sub('([^a-z])secreto', r'\1 secreto',text, flags=re.I)
        text = re.sub('([^a-z])reservado', r'\1 reservado',text, flags=re.I)
        text = re.sub('petr[óòoôõ\w]leo', r'petróleo',text, flags=re.I)
        text = re.sub('econ[óòoôõ\w]mic([ao]s?[^a-z])', r'econômic\1',text, flags=re.I)
        text = re.sub('rep[úùuû]b(u|li)ca', r'república',text, flags=re.I)
        text = re.sub('delega[cç][áàaâã]o', r'delegação',text, flags=re.I)
        text = re.sub('na[çc][óòoôõ]es', r'nações',text, flags=re.I)
        text = re.sub('([^a-z])est[áàaâã]o([^a-z])', r'\1estão\2',text, flags=re.I)
        text = re.sub('([^a-z])per[ìíiî]odo([^a-z])', r'\1período\2',text, flags=re.I)
        text = re.sub('retransm[íìiîl0-9]ss?[áàaâã]o', r'retransmissão',text, flags=re.I)
        text = re.sub('import[áàaâã]ncia', r'importância',text, flags=re.I)
        text = re.sub('negoc[íìiîl0-9]a[cç][óòoôõ\w]es', r'negociações',text, flags=re.I)
        text = re.sub('([^a-z])m[íìiîl0-9]ss[áàaâã]o([^a-z])', r'\1missão\2',text, flags=re.I)
        text = re.sub('urgent[[íìiîl0-9]]ss[íìiîl0-9]mo', r'urgentíssimo',text, flags=re.I)
        text = re.sub('bras[ìiîl0-9]l([b-df-hj-z0-9])', r'brasil \1',text, flags=re.I)
        text = re.sub('assun[çc][áàaâã]o', r'assunção',text, flags=re.I)
        text = re.sub('dec[l1]ara[cç][óòoôõ]es', r'declarações',text, flags=re.I)
        text = re.sub('([^a-z])ostens\wvo', r'\1 ostensivo',text, flags=re.I)
        text = re.sub('conversa[cç][óòoôõ]es', r'conversações',text, flags=re.I)
        text = re.sub('condi[cç][óòoôõ]es', r'condições',text, flags=re.I)
        text = re.sub('brasi ([^l])', r'brasil \1',text, flags=re.I)
        text = re.sub('pre[saã][íìiîl0-9]?dente', r'presidente',text, flags=re.I)
        text = re.sub('pr[íìiîl0-9]nc[íìiîl0-9]p[íìiîl0-9]o(s?)', r'princípio\1',text, flags=re.I)
        text = re.sub('[íìiîl0-9]nstru[cç][óòoôõ\w]es', r'instruções',text, flags=re.I)
        text = re.sub('terr[íìiîl0-9]t[óòoôõ]rio(s?)', r'território\1',text, flags=re.I)
        text = re.sub('seguranca', r'segurança',text, flags=re.I)
        text = re.sub('neg[óòoôõ]c[íìiîl0-9]os', r'negócios',text, flags=re.I)
        text = re.sub('[íìiîl0-9]nforma[cç][óòoôõ]es', r'informações',text, flags=re.I)
        text = re.sub('sess[áàaâã]o', r'sessão',text, flags=re.I)
        text = re.sub('([^a-z])ultimos([^a-z])', r'\1últimos\2',text, flags=re.I)
        text = re.sub('([^a-z])jap[áàaâã]o([^a-z])', r'\1japão\2',text, flags=re.I)
        text = re.sub('([^a-z])jo[áàaãâ]o([^a-z])', r'\1joão\2',text, flags=re.I)
        text = re.sub('part[íìiîl0-9]c[íìiîl0-9]pa[cça][áàaãâ]o', r'participação',text, flags=re.I)
        text = re.sub('([^a-z])op[íìiîl0-9]n[íìiîl0-9][áàaâã]o([^a-z])', r'\1opinião\2',text, flags=re.I)
        text = re.sub('([^a-z])v[áàaâã]rios([^a-z])', r'\1vários\2',text, flags=re.I)
        text = re.sub('t[éèeê]cn[íìiîl0-9]c([ao]s?[^a-z])', r'técnic\1',text, flags=re.I)
        text = re.sub('([^a-z])[áàaâã]guas([^a-z])', r'\1águas\2',text, flags=re.I)
        text = re.sub('reso[l1]u[cç][áàaâã]o', r'resolução',text, flags=re.I)
        text = re.sub('dec[l1]ara[cç][áàaâã]o', r'declaração',text, flags=re.I)
        text = re.sub('[íìiîl0-9]ndepend[éèeê]ncia', r'independência',text, flags=re.I)
        text = re.sub('([^a-z])ara[úùuû]jo([^a-z])', r'\1Araújo\2',text, flags=re.I)
        text = re.sub('([^a-z])un[íìiîl0-9][áàaâã]o([^a-z])', r'\1união\2',text, flags=re.I)
        text = re.sub('[íìiîl0-9]nforma[cç][áàaâã]o', r'informação',text, flags=re.I)
        text = re.sub('quest[óòoôõ]es', r'questões',text, flags=re.I)
        text = re.sub('([^a-z])regi[áàaâã0-9]o([^a-z])', r'\1região\2',text, flags=re.I)
        text = re.sub('portugu[èéê\w]s([^ae])', r'português\1',text, flags=re.I)
        text = re.sub('d[íìiîl0-9]f[íìiî]c[íìiîl0-9]l([^a-z])', r'difícil\1',text, flags=re.I)
        text = re.sub('([^a-z])precos([^a-z])', r'\1preços\2',text, flags=re.I)
        text = re.sub('([^a-z])conf[íìiîl0-9]denc[íìiîl0-9]a[il1]([^a-z])', r'\1 confidencial\2',text, flags=re.I)
        text = re.sub('pos[íìiîl0-9][cç][óòôoõ]es', r'posições',text, flags=re.I)
        text = re.sub('pres[íìiîl0-9]d[éèeê]ncia([^a-z])', r'presidência\1',text, flags=re.I)
        text = re.sub('constru[cç][áàaâã]o', r'construção',text, flags=re.I)
        text = re.sub('relat[óòoôõ]rio', r'relatório',text, flags=re.I)
        text = re.sub('reuníìiîl0-9[óòoôõ]es', r'reuniões',text, flags=re.I)
        text = re.sub('([^a-z])ur[áàaâã\w]nio([^a-z])', r'\1urânio\2',text, flags=re.I)
        text = re.sub('preocupa[cç][áàaâã]o', r'preocupação',text, flags=re.I)
        text = re.sub('([^a-z])cor[éèeê][íìiîl0-9]a([^a-z])', r'\1coreia\2',text, flags=re.I)
        text = re.sub('organ[íìiîl0-9]za[cç][áàaâã]o', r'organização',text, flags=re.I)
        text = re.sub('sov[íìiîl0-9][éèeê]t[íìiîl0-9]c([ao]s?)', r'soviétic\1',text, flags=re.I)
        text = re.sub('necess[áàaâã]r[íìiîl0-9]([ao]s?[^a-z])', r'necessári\1',text, flags=re.I)
        text = re.sub('([^a-z])c[áàaâã]mara([^a-z])', r'\1câmara\2',text, flags=re.I)
        text = re.sub('func[íìiîl0-9]on[áàaâã]ri([ao]s?)', r'funcionári\1',text, flags=re.I)
        text = re.sub('produ[cç][áàaâã]o', r'produção',text, flags=re.I)
        text = re.sub('sa[úùuû]de', r'saúde',text, flags=re.I)
        text = re.sub('cr[íìiîl0-9]a[cç][áàaâã]o', r'criação',text, flags=re.I)
        text = re.sub('aud[íìiîl0-9][éèeê]ncia', r'audiência',text, flags=re.I)
        text = re.sub('([^a-z])[íìiîl0-9]t[áàaâã]lia([^a-z])', r'\1itália\2',text, flags=re.I)
        text = re.sub('or[íìiîl0-9]enta[cç][áàaâã]o', r'orientação',text, flags=re.I)
        text = re.sub('([^a-z])rea[cç][áàaâã]o([^a-z])', r'\1reação\2',text, flags=re.I)
        text = re.sub('([^a-z])[íìiîl0-9]nd[íìiîl0-9]ce([^a-z])', r'\1índice\2',text, flags=re.I)
        text = re.sub('admin[íìiîl0-9]stra[cç][áàaâã]o', r'administração',text, flags=re.I)
        text = re.sub('([^\s])azeredo', r'\1 Azeredo',text, flags=re.I)
        text = re.sub('exporta[cç][áàaâã]o', r'exportação',text, flags=re.I)
        text = re.sub('opera[cç][áàaâã]o', r'operação',text, flags=re.I)
        text = re.sub('hip[óòoôõ]tese', r'hipótese',text, flags=re.I)
        text = re.sub('real[íìiîl0-9]za[cç][áàaâã]o', r'realização',text, flags=re.I)
        text = re.sub('tel[èéeê]g\wama', r'telegrama',text, flags=re.I)
        text = re.sub('ele[íìiîl0-9][cç][óòoôõ\w]es', r'eleições',text, flags=re.I)
        text = re.sub('gest[áàaâã]o', r'gestão',text, flags=re.I)
        text = re.sub('delega[cç][óòoôõ]es', r'delegações',text, flags=re.I)
        text = re.sub('europ[èéeê][íìiîl0-9]a', r'europeia',text, flags=re.I)
        text = re.sub('transfer[èéeê]ncia', r'transferência',text, flags=re.I)
        text = re.sub('montev[íìiîl0-9]d[èéeê]u', r'montevidéu',text, flags=re.I)
        text = re.sub('concess[áàaâã]o', r'concessão',text, flags=re.I)
        text = re.sub('exporta[cç][óòoôõ\w]es', r'exportações',text, flags=re.I)
        text = re.sub('bras[íìiîl0-9]l—', r'brasil -',text, flags=re.I)
        text = re.sub('comun[íìiîl0-9]ca[cç][áàaâã]o', r'comunicação',text, flags=re.I)
        text = re.sub('([^a-z])g[óòoôõ0-9]v[èéeê]rno([^a-z])', r'\1governo\2',text, flags=re.I)
        text = re.sub('esforcos', r'esforços',text, flags=re.I)
        text = re.sub('atua[cç][áàaâã]o', r'atuação',text, flags=re.I)
        text = re.sub('negoc[íìiîl0-9]a[cç][áàaâã]o', r'negociação',text, flags=re.I)
        text = re.sub('[íìiîl0-9]mpress[áàaâã]o', r'impressão',text, flags=re.I)
        text = re.sub('na[cç][áàaâã]o', r'nação',text, flags=re.I)
        text = re.sub('alem[áàaâã]o', r'alemão',text, flags=re.I)
        text = re.sub('[íìiîl0-9]nterven[cç][áàaâã]o', r'intervenção',text, flags=re.I)
        text = re.sub('[íìiîl0-9]nforma[cç].es', r'informações',text, flags=re.I)
        text = re.sub('aprova[cç][áàaâã]o', r'aprovação',text, flags=re.I)
        text = re.sub('[íìiîl0-9]ndica[cç][áàaâã]o', r'indicação',text, flags=re.I)
        text = re.sub('d[èéeê]tente', r'détente',text, flags=re.I)
        text = re.sub('opos[íìiîl0-9][cç][áàaâã]o', r'oposição',text, flags=re.I)
        text = re.sub('equ[íìiîl0-9]l[íìiî]brio', r'equilíbrio',text, flags=re.I)
        text = re.sub('camili[óòoôõ]n', r'camilión',text, flags=re.I)
        text = re.sub('ed[íìiîl0-9][cç][áàaâã]o', r'edição',text, flags=re.I)
        text = re.sub('colabora[cç][áàaâã]o', r'colaboração',text, flags=re.I)
        text = re.sub('pos[íìiîl0-9][cç][áàaâã]o', r'posição',text, flags=re.I)
        text = re.sub('comun[íìiîl0-9]ca[cç][óòoôõ]es', r'comunicações',text, flags=re.I)
        text = re.sub('([^a-z])\wuest[áàaâã]o([^a-z])', r'\1questão\2',text, flags=re.I)
        text = re.sub('expans[áàaâã]o', r'expansão',text, flags=re.I)
        text = re.sub('([^a-z])[óòoôõ]rg[aáàâã]o(s?[^a-z])', r'\1órgão\2',text, flags=re.I)
        text = re.sub('explora[cç][áàaâã]o', r'exploração',text, flags=re.I)
        text = re.sub('([^a-z])di[áàaâã]rio([^a-z])', r'\1diário\2',text, flags=re.I)
        text = re.sub('bras[ìiîl0-9]la([a-z])', r'brasil a\1',text, flags=re.I)
        text = re.sub('urg[èéeê]nc[íìiîl0-9]a', r'urgência',text, flags=re.I)
        text = re.sub('just[íìiîl0-9][cç]a', r'justiça',text, flags=re.I)
        text = re.sub('serv[íìiîl0-9][cç]o', r'serviço',text, flags=re.I)
        text = re.sub('d[íìiîl0-9]v[íìiîl0-9]s[áàaâã]o', r'divisão',text, flags=re.I)
        text = re.sub('(\/?)brase[hnm]b', r'\1brasemb',text, flags=re.I)
        text = re.sub('te[l1]egr[áàaâã][^f\W]a', r'telegrama',text, flags=re.I)
        text = re.sub('[íìiîl0-9]mporta[cç][áàaâã]o', r'importação',text, flags=re.I)
        text = re.sub('serv[íìiîl0-9]cos', r'serviços',text, flags=re.I)
        text = re.sub('echeverr[íìî\w]a', r'echeverría',text, flags=re.I)
        text = re.sub('([^a-z])ac[óòoôõ]rdo([^a-z])', r'\1acordo\2',text, flags=re.I)
        text = re.sub('c[íìiîl0-9][èéeê]ncia', r'ciência',text, flags=re.I)
        text = re.sub('combust[íìî\w]vel', r'combustível',text, flags=re.I)
        text = re.sub('([^d\W\d]{3})a[mn][èéeê]r\wca[mn]', r'\1-american',text, flags=re.I)
        text = re.sub('inter-american', r'interamerican',text, flags=re.I)
        text = re.sub('lat[íìiîl0-9]no-?american([ao]s?[^a-z])', r'latino-american\1',text, flags=re.I)
        text = re.sub('t[óòoôõ]qu[íìiîl0-9]o', r'tóquio',text, flags=re.I)
        text = re.sub('rela[\w]ões', r'relações',text, flags=re.I)
        text = re.sub('([^a-z])clar[íìiî]n([^a-z])', r'\1clarín\2',text, flags=re.I)
        text = re.sub('telev[íìiîl0-9]s[áàaâã]o', r'televisão',text, flags=re.I)
        text = re.sub('man[íìiîl0-9]festa[cç][áàaâã]o', r'manifestação',text, flags=re.I)
        text = re.sub('[íìiîl0-9]mporta[cç][óòoôõ\w]es', r'importações',text, flags=re.I)
        text = re.sub('dafontoura', r'da fontoura',text, flags=re.I)
        text = re.sub('m[íìiîl0-9]ss[óòoôõ]es', r'missões',text, flags=re.I)
        text = re.sub('japon[èéeê]s([^a-z])', r'japonês\1',text, flags=re.I)
        text = re.sub('exped[íìiîl0-9]doem', r'expedido em',text, flags=re.I)
        text = re.sub('d[íìiîl0-9]re[cç][áàaâã]o', r'direção',text, flags=re.I)
        text = re.sub('compet[èéeê]nc[íìiîl0-9]a', r'competência',text, flags=re.I)
        text = re.sub('[íìiîl0-9]nstala[cç][áàaâã]o', r'instalação',text, flags=re.I)
        text = re.sub('jur[íìiîl0-9]d[íìiîl0-9]c([ao]s?[^a-z])', r'jurídic\1',text, flags=re.I)
        text = re.sub('restr[íìiîl0-9][cç][óòoôõ]es', r'restrições',text, flags=re.I)
        text = re.sub('esforco', r'esforço',text, flags=re.I)
        text = re.sub('br[íìiîl0-9]t[áàaâã\w]nic([ao]s?)', r'britânic\1',text, flags=re.I)
        text = re.sub('aproxima[cç][áàaâã]o', r'aproximação',text, flags=re.I)
        text = re.sub('plen[áàaâã]rio', r'plenário',text, flags=re.I)
        text = re.sub('const[íìiîl0-9]tu[íìiîl0-9][cç][áàaâã]o', r'constituição',text, flags=re.I)
        text = re.sub('manuten[cç][áàaâã]o', r'manutenção',text, flags=re.I)
        text = re.sub('c[íìiî]rculos', r'círculos',text, flags=re.I)
        text = re.sub('d[íìiîl0-9]vulga[cç][áàaâã]o', r'divulgação',text, flags=re.I)
        text = re.sub('exter[íìiîl0-9]\wres', r'exteriores',text, flags=re.I)
        text = re.sub('vota[cç][áàaâã]o', r'votação',text, flags=re.I)
        text = re.sub('alem[áàaâã]es', r'alemães',text, flags=re.I)
        text = re.sub('subs[íìiîl0-9]dio(s?)', r'subsídio\1',text, flags=re.I)
        text = re.sub('([^a-z])n[ou]c\wea\w([^a-z])', r'\1nuclear\2',text, flags=re.I)     
        text = re.sub('b[íìiîl0-9]ll[íìiîl0-9]\wns', r'billions',text, flags=re.I)
        text = re.sub('manifesta[cç][óòoôõ]es', r'manifestações',text, flags=re.I)
        text = re.sub('at[óòoôõ]m[íìiîl0-9]c([ao]s?[^a-z])', r'atômic\1',text, flags=re.I)
        text = re.sub('tecnol[óòoôõ]g[íìiîl0-9]c([ao]s?[^a-z])', r'tecnológic\1',text, flags=re.I)
        text = re.sub('a[cç][uú]car([^a-z])', r'açúcar\1',text, flags=re.I)
        text = re.sub('concess[óòoôõ]es', r'concessões',text, flags=re.I)
        text = re.sub('sald[íìiî]var', r'saldívar',text, flags=re.I)
        text = re.sub('conversa[cç][óòoôõ]es', r'conversações',text, flags=re.I)
        text = re.sub('amaz[óòoôõ]n[íìiîl0-9]a', r'amazônia',text, flags=re.I)
        text = re.sub('secretarygeneral', r'secretary-general',text, flags=re.I)
        text = re.sub('l[íìiîl0-9]berta[cç][áàaâã]o', r'libertação',text, flags=re.I)
        text = re.sub('suspens[áàaâã]o', r'suspensão',text, flags=re.I)
        text = re.sub('h[íìiîl0-9]drel[èéeê]tric([ao]s?[^a-z])', r'hidrelétric\1',text, flags=re.I)
        text = re.sub('revolu[cç][áàaâã]o', r'revolução',text, flags=re.I)
        text = re.sub('descolon[íìiîl0-9]za[cç][áàaâã]o', r'descolonização',text, flags=re.I)
        text = re.sub('sov[íìiîl0-9][èéeê]tic([ao]s?[^a-z])', r'soviétic\1',text, flags=re.I)
        text = re.sub('rod[èéeê]s[íìiîl0-9]a([^a-z])', r'rodésia\1',text, flags=re.I)
        text = re.sub('revo[l1]u[cç][áàaâã]o', r'revolução',text, flags=re.I)
        text = re.sub('atl[áàaâã]nt[íìiîl0-9]c([ao]s?)', r'atlântic\1',text, flags=re.I)
        text = re.sub('plut[óòoôõ]n[íìiîl0-9]o', r'plutônio',text, flags=re.I)
        text = re.sub('subst[íìiîl0-9]tu[íìiîl0-9][cç][áàaâã]o', r'substituição',text, flags=re.I)
        text = re.sub('[íìiîl0-9]ta[íìiîl0-9]puh', r'itaipu',text, flags=re.I)
        text = re.sub('([^a-z])l[íìiîl0-9]bano([^a-z])', r'\1líbano\2',text, flags=re.I)
        text = re.sub('prog\wess?o', r'progresso',text, flags=re.I)
        text = re.sub('hem[íìiîl0-9]sf[èéeê]rio', r'hemisfério',text, flags=re.I)
        text = re.sub('ant[áàaâã]rt[íìiîl0-9]c([ao]s?[^a-z])', r'antártic\1',text, flags=re.I)
        text = re.sub('([^a-z])ameaca([^a-z])', r'\1ameaça\2',text, flags=re.I)
        text = re.sub('pol[óòoôõ]n[íìiîl0-9]a', r'polônia',text, flags=re.I)
        text = re.sub('promo[cç][áàaâã]o', r'promoção',text, flags=re.I)
        text = re.sub('([^a-z])rom[èéeê]nia([^a-z])', r'\1romênia\2',text, flags=re.I)
        text = re.sub('([^a-z])n[ou]cl[èéeê]b\w[áàaâã]s([^a-z])', r'\1nuclebrás\2',text, flags=re.I)
        text = re.sub('d[íìiîl0-9]p[l1]om[áàaâã]t[íìiîl0-9]c([ao]s?[^a-z])', r'diplomátic\1',text, flags=re.I)
        text = re.sub('bras[íìiîl0-9]l[èéeê][nñ]([ao]s?[^a-z])', r'brasileñ\1',text, flags=re.I)
        text = re.sub('([^\w])asunc[íìiîl0-9][óòoôõ]n([^\w])', r'\1asunción\2',text, flags=re.I) 
        text = re.sub('([^\w])[íìiîl0-9]nclus[áàaâã]o([^\w])', r'\1inclusão\2',text, flags=re.I)
        text = re.sub('te[l1]egr[áàaâã]fic([ao]s?[^a-z])', r'telegráfic\1',text, flags=re.I)
        text = re.sub('organ[íìiîl0-9]zac[íìiîl0-9][óòoôõ]n(e?s?[^a-z])', r'organización',text, flags=re.I)
        text = re.sub('plen[íìiîl0-9]potenc[íìiîl0-9][áàaâã]ri([ao]s?[^a-z])', r'plenipotenciári\1',text, flags=re.I)
        text = re.sub('([^\w])nogu[èéê]s([^\w])', r'\1nogués\2',text, flags=re.I)
        text = re.sub('([^\w])voss[èéeê]ncia([^\w])', r'\1vossência\2',text, flags=re.I)
        text = re.sub('ass[íìiîl0-9]st[èéeê]nc[íìiîl0-9]a([^li])', r'assistência\1',text, flags=re.I)
        text = re.sub('depe\wde[hnm]', r'dependen',text, flags=re.I)
        text = re.sub('l9([0-9][0-9])', r'19\1',text, flags=re.I)        
        text = re.sub('nortesu[l1]',r'norte-sul',text, flags=re.I)
        text = re.sub('sulafr[íìiîl0-9]c',r'sul-afric',text, flags=re.I)
        text = re.sub('agr[íìiîl0-9]co[l1]', r'agrícol',text, flags=re.I)
        text = re.sub('navega[cç][áàaâã]o', r'navegação',text, flags=re.I)
        text = re.sub('de[l1]egac[íìiî]on', r'delegación',text, flags=re.I)
        text = re.sub('[íìiî]ntegrac[íìiî][óòoôõ]n', r'integración',text, flags=re.I)
        text = re.sub('organ[íìiî]zac[íìiî][óòoôõ]on', r'organización',text, flags=re.I)
        text = re.sub('assoc[íìiî]a[cç][áàaâã]o', r'associação',text, flags=re.I)
        text = re.sub('[íìiî]ntegra[cç][áàaâã]o', r'integração',text, flags=re.I)
        text = re.sub('conven[cç][áàaâã]o', r'convenção',text, flags=re.I)
        text = re.sub('pro[l1][íìiî]fera[cç][áàaâã]o', r'proliferação',text, flags=re.I)
        text = re.sub('([^a-z])n[ou]\wlea\w([^a-z])', r'\1nuclear\2',text, flags=re.I)
        text = re.sub('p\w\wít[íìiîl0-9]c([ao]s?[^a-z])', r'polític\1',text, flags=re.I)
        text = re.sub('pol\wt\wc([ao]s?[^a-z])', r'polític\1',text, flags=re.I)
        text = re.sub('([^a-z])s\w\wve\wra([^a-z])', r'\1silveira\2',text, flags=re.I)
        text = re.sub('([^a-z])s[íìiîl0-9]\wve[íìiîl0-9]\w\w([^a-z])', r'\1silveira\2',text, flags=re.I)
        text = re.sub('(silveira)(\w)', r'\1 \2',text, flags=re.I)
        text = re.sub('([^\s-])(silveira)', r'\1 \2',text, flags=re.I)
        text = re.sub('([^a-z])azere\wo([^a-z])', r'\1azeredo\2',text, flags=re.I)
        text = re.sub('([^a-z])a\weredo([^a-z])', r'\1azeredo\2',text, flags=re.I)
        text = re.sub('(azeredo)(\w)', r'\1 \2',text, flags=re.I)
        text = re.sub('([^\s-])(azeredo)', r'\1 \2',text, flags=re.I)
        text = re.sub('([^a-z])ge[íìiîl0-9]se\w([^a-z])', r'\1geisel\2',text, flags=re.I)
        text = re.sub('([^\s-])(geisel)', r'\1 \2',text, flags=re.I)
        text = re.sub('([^a-z])b\wa[^sz]il([^a-z])', r'\1brasil\2',text, flags=re.I)
        text = re.sub('([^a-z])b\was\wl([^a-z])', r'\1brasil\2',text, flags=re.I)
        text = re.sub('([^a-z])a[^Wr]gola(n?[ao]s?[^a-z])', r'\1angola\2',text, flags=re.I)
        text = re.sub('([^a-z])ango\wa(n?[ao]s?[^a-z])', r'\1angola\2',text, flags=re.I)
        text = re.sub('([^a-z])k\wss\w\wger([^a-z])', r'\1Kissinger\2',text, flags=re.I)
        text = re.sub('([^a-z])ki\w\winger([^a-z])', r'\1Kissinger\2',text, flags=re.I)
        text = re.sub('([^\s-])k\wss\wnger', r'\1 Kissinger',text, flags=re.I)
        text = re.sub('([^a-z])a\wer\wcan([ao]s?[^a-z])', r'\1american\2',text, flags=re.I)
        text = re.sub('(\s)(merican)([ao]s?[^a-z])', r'\1a\2\3',text, flags=re.I)
        text = re.sub('([^a-z])d\wre[íìiîl0-9]to([^a-z])', r'\1direito\2',text, flags=re.I)
        text = re.sub('([^a-z])d[íìiîl0-9]\w?e[íìiîl0-9]\wo([^a-z])', r'\1direito\2',text, flags=re.I)
        text = re.sub('([^\s-])(direito)', r'\1 \2',text, flags=re.I)
        text = re.sub('d[íìiîl0-9]re[íìiîl0-9]tos[^\s][hmn]umanos', r'direitos humanos',text, flags=re.I)
        text = re.sub('[íìiîl0-9]\wdependen', r'independen',text, flags=re.I)
        text = re.sub('depe\wdê\wc[íìiîl0-9]a', r'dependência',text, flags=re.I)
        text = re.sub('au\wono\w[íìiîl0-9]a', r'autonomia',text, flags=re.I)
        text = re.sub('au\wo\wom\wa', r'autonomia',text, flags=re.I)
        text = re.sub('não-?[íìiîl0-9]\wte\w\wenç\w\w', r'não-intervenção',text, flags=re.I)
        text = re.sub('não-?[íìiîl0-9]nter\wen\wão', r'não-intervenção',text, flags=re.I)
        text = re.sub('nac\wonal', r'nacional',text, flags=re.I)
        text = re.sub('nac[íìiîl0-9]o\wal', r'nacional',text, flags=re.I)
        text = re.sub('dete\wm\wna\wão([^a-z])', r'determinação\1',text, flags=re.I)
        text = re.sub('determ[íìiîl0-9]naç\wo([^a-z])', r'determinação \1',text, flags=re.I)
        text = re.sub('au\w\w-?determ\w\wa\w\wo', r'auto-determinação',text, flags=re.I)
        text = re.sub('su\wve\wsão', r'subversão',text, flags=re.I)
        text = re.sub('comu\w[íìiîl0-9]s', r'comunis',text, flags=re.I)
        text = re.sub('ant[íìiîl0-9]comu', r'anti-comu',text, flags=re.I)
        text = re.sub('g\w[íìiîl0-9]n\w-?b\ws\wau', r'guiné-bissau',text, flags=re.I)
        text = re.sub('gu\w\wé-?\wissau', r'guiné-bissau',text, flags=re.I)
        text = re.sub('mo\wamb\w\wue', r'moçambique',text, flags=re.I)
        text = re.sub('go\wbery', r'golbery',text, flags=re.I)
        text = re.sub('\w\wama\waty', r'itamaraty',text, flags=re.I)
        text = re.sub('([^a-z])cr\wmm\wns([^a-z])', r'\1crimmins\2',text, flags=re.I)
        text = re.sub('([^a-z])d\wt\wnte([^a-z])', r'\1détente\2',text, flags=re.I)
        text = re.sub('([^a-z])dé\we\wte([^a-z])', r'\1détente\2',text, flags=re.I)
        text = re.sub('([^a-z])Cal\wag\wa\w([^a-z])', r'\1callaghan\2',text, flags=re.I)
        text = re.sub('([^a-z])Ca\wlag\wa\w([^a-z])', r'\1callaghan\2',text, flags=re.I)
        text = re.sub('(\w[\s])aa([\s]\w)',r'\1à\2',text, flags=re.I)
        text = re.sub('(\w[\s])aas([\s]\w)',r'\1às\2',text, flags=re.I)
        text = re.sub('(\w[\W])jah([\W]\w)',r'\1já\2',text, flags=re.I)
        text = re.sub('(\w[\W])oue([\W]\w)',r'\1que\2',text, flags=re.I)
        text = re.sub('(\w[\W])t0([\W]\w)',r'\1to\2',text, flags=re.I)
        text = re.sub('(\w[\W])estah([\W]\w)',r'\1está\2',text, flags=re.I)
        text = re.sub('(\w[\W])ateh([\W]\w)',r'\1até\2',text, flags=re.I)
        text = re.sub('([^a-z])tamb[èéeê]m([^a-z])', r'\1também\2',text, flags=re.I)
        text = re.sub('a0', r'ao',text, flags=re.I)
        text = re.sub('([^\w])al[èéeê]m([^\w])', r'\1além\2',text, flags=re.I)
        text = re.sub('([^a-z])por[èéê]m([^a-z])', r'\1porém\2',text, flags=re.I)
        text = re.sub('([^a-z])[úùuû]nic([ao]s?[^a-z])', r'\1únic\2',text, flags=re.I)
        text = re.sub('([^a-z])tres([^a-z])', r'\1três \2',text, flags=re.I)
        text = re.sub('pos[íiìî0-9][cç][áàaâã]o([^a-z])', r'posição\1',text, flags=re.I)
        text = re.sub('n[íiìîl0-9]ve[l1]([^\w])', r'nível\1',text, flags=re.I)
        text = re.sub('car[áàaâã]ter', r'caráter',text, flags=re.I)
        text = re.sub('([^a-z])poss[\w]ve[l0-9]([^a-z])', r'\1possível\2',text, flags=re.I)
        text = re.sub('s[íìiîl0-9]tua[çc][áàaâã]o', r'situação',text, flags=re.I)
        text = re.sub('dec[íìiîl0-9]s[áàaâã]o', r'decisão',text, flags=re.I)
        text = re.sub('atrav[èéeê]s([^a-z])', r'através\1',text, flags=re.I)
        text = re.sub('([^a-z])id[èéeê][íìiî]a(s?[^a-z])', r'\1ideia\2',text, flags=re.I)
        text = re.sub('([^a-z])pr[óòoôõo]xim([ao]s?[^a-z])', r'\1próxim\2',text, flags=re.I)
        text = re.sub('solu[cç][áàaâã]o', r'solução',text, flags=re.I)
        text = re.sub('mi[l1]h[óòôoõ]es', r'milhões',text, flags=re.I)
        text = re.sub('d[òóoôõ][l1]ares', r'dólares',text, flags=re.I)
        text = re.sub('ocas[íìiîl0-9][áàaâã]o', r'ocasião',text, flags=re.I)
        text = re.sub('([^a-z])serao([^a-z])', r'\1serão\2',text, flags=re.I)
        with open(os.path.join(path_out,txt), 'w', encoding='utf-8') as f:
            txt = f.write(text)
        text = text.split()
        text = [t.strip(string.punctuation) for t in text]
        freqdist = Counter(text)
        freqtotal.update(freqdist)


### exploratory analysis

In [5]:
sorted_freqtotal = sorted(freqtotal.items(), key=operator.itemgetter(1), reverse=True)

In [6]:
len(sorted_freqtotal)

1108044

In [7]:
sorted_freqtotal[0:10]

[('de', 608175),
 ('', 402021),
 ('que', 225606),
 ('do', 208707),
 ('the', 183720),
 ('da', 183457),
 ('em', 158995),
 ('of', 116212),
 ('no', 101836),
 ('para', 101502)]

### lists not identified corpus-specific words
Makes a list of words that weren't dictionarized at enchant, but are important for this corpus. The list might be used for other tasks.

In [11]:
lista = ['vg', 'brasemb', 'eua', 'onu', 'oea', 'ptvg', 'bsb', 'df', 'rfa', 'ue', 'urss', 'delbrasonu', 'zczc', 'aiea', 'cee', 
         'stroessner', 'brazext', 'gatt', 'aditel', 'ueki', 'unctad', 'velloso', 'tlatelolco', 'debernardi', 'nogues', 'eletrobrás', 
         'nnnn', 'itaipú', 'araújo', 'oas', 'mpla', 'crimmins', 'rpa', 'cavalcanti', 'mello', 'desptel', 'urenco', 'coréia', 'dti', 
         'simonsen', 'alalc', 'opep', 'tnp', 'dm', 'banzer', 'ecosoc', 'cacex', 'tv', 'videla', 'acaray', 'petrobrás', 'blanco', 
         'européia', 'mre', 'genscher', 'unep', 'varig', 's/a', 'iaea', 'shigeaki', 'camilión', 'rpc', 'cidh', 'allara', 'emb', 
         'akasaka', 'olp', 'sauvagnargues', 'golbery', 'netto', 'guzzetti', 'ltda', 'pnud', 'mdb', 'bronstein', 'ncms', 'refdesptel', 
         'fnla', 'fmi', 'echeverría', 'todman', 'sni', 'byelorussian', 'fukuda', 'braziliense', 'oua', 'baptista', 'rda', 'ccei', 
         'pcp', 'braspetro', 'ccd', 'cnpq', 'consbras', 'jamieson', 'barboza', 'antónio', 'cvrd', 'opec', 'dasp', 'infce', 'arana', 
         'stoel', 'bnde', 'saldívar', 'rosalynn', 'unitar', 'braseme', 'cepal', 'ee.uu', 'seplan', 'sudene', 'scheel', 'burnham', 
         'norte-sul', 'salom', 'npt', 'mobral', 'frelimo', 'prg', 'pérez', 'sapena', 'portella', 'vatemb', 'cnen', 'perú', 
         'paulinelli', 'machel', 'cds', 'usiminas', 'rowlands', 'baena', 'bordaberry', 'pnd', 'bermudez', 'mfa', 'ocde', 'ibc', 
         'rovira', 'komoto', 'cdi', 'sette', 'magalhaes', 'helder', 'iguazu', 'futscher', 'undp', 'ayacucho', 'iguazú', 'guilhon', 
         'país-chave', 'auto-determinação', 'não-alinhados', 'não-intervenção', 'détente', 'terceiro-mundo', 
         'não-proliferação', 'rapprochement', 'descolonização', 'dsn', 'csn', 'meio-ambiente', 'brasil-alemanha', 'vernon', 
         'walters', 'gibson', 'esg', 'gromyko', 'herzog', 'callaghan', 
         'cnmf', 'nuclebrás', 'mw', 'bmw', 'grã-cruz', 'infra-estrutura', 'brasil-eua', 'matéria-prima', 'matérias-primas', 
         'assembleia-geral', 'mão-de-obra', 'secreto-urgentíssimo', 'torrijos', 'ossorio', 'países-membros', 'delbrasgen', 
         'delbrasupa', 'cdh', 'ptfim', 'erasemb', 'respdesptel', 'ceesi', 'brasil-rpa', "brazil's", "people's", "committee's", 
         "america's", "country's", "secretary-general's", 'coopg', 'minulado']

### creates file to check text errors
Creates a file with a list of all errors and respective frequencies

In [8]:
with open('saida.txt', 'w') as f:
    for item in sorted_freqtotal:
        f.write('{} --> {}'.format(item[0], item[1]))

In [9]:
print(enchant.list_languages())

['fr_CH', 'fr_FR', 'fr_BE', 'fr_CA', 'fr_LU', 'fr_MC', 'es_CL', 'es_SV', 'pt_PT', 'es_PR', 'es_CO', 'de_LU', 'fr', 'es_ES', 'es_VE', 'es_BO', 'en_US', 'de_DE', 'es_US', 'es_MX', 'es_HN', 'es_AR', 'es_PY', 'es_PE', 'en_GB', 'es_CU', 'it_CH', 'de_BE', 'es_CR', 'pt_BR', 'es_GT', 'es_PA', 'es_UY', 'es_EC', 'es_NI', 'es_DO', 'it_IT', 'en', 'en_CA']


In [10]:
dic_us = enchant.Dict("en_US")
dic_gb = enchant.Dict("en_GB")
dic_es = enchant.Dict("es_ES")
dic_pt = enchant.Dict("pt_BR")
dic_pt2 = enchant.Dict("pt_PT")
dic_fr = enchant.Dict("fr_FR")
dic_al = enchant.Dict("de_BE")

In [ ]:
sorted_freqtotal_nondic = []
for word, count in sorted_freqtotal:
    try:
        if not dic_us.check(word.title()) \
        and not dic_pt.check(word.title()) \
        and not dic_es.check(word.title()) \
        and not dic_fr.check(word.title()) \
        and not dic_al.check(word.title()) \
        and not dic_gb.check(word.title()) \
        and not dic_pt2.check(word.title()) \
        and word.lower() not in lista:
            sorted_freqtotal_nondic.append((word,count))
    except: 
        pass

In [ ]:
sorted_freqtotal_nondic[0:10]

In [14]:
with open('saida_nondic.txt', 'w') as f:
    for item in sorted_freqtotal_nondic:
        f.write('{} --> {} ; '.format(item[0], item[1]))